In [1]:
%reload_ext autoreload
%autoreload 2

from pyeed.core.proteinsequence import ProteinSequence
from pyeed.blast.blast import pblast

In [2]:
seq = ProteinSequence.from_ncbi("NP_001287541.1")

In [3]:
print(seq)

ProteinSequence
├── id = NP_001287541.1
├── name = aldolase 1, isoform M [Drosophila melanogaster]
├── amino_acid_sequence = MTTYFNYPSKELQDELREIAQKIVAPGKGILAADESGPTMGKRLQDIGVENTEDNRRAYRQLLFSTDPKLAENISGVILFHETLYQKADDGTPFAEILKKKGIILGIKVDKGVVPLFGSEDEVTTQGLDDLAARCAQYKKDGCDFAKWRCVLKIGKNTPSYQSILENANVLARYASICQSQRIVPIVEPEVLPDGDHDLDRAQKVTETVLAAVYKALSDHHVYLEGTLLKPNMVTAGQSAKKNTPEEIALATVQALRRTVPAAVTGVTFLSGGQSEEEATVNLSAINNVPLIRPWALTFSYGRALQASVLRAWAGKKENIAAGQNELLKRAKANGDAAQGKYVAGSAGAGSGSLFVANHAY
├── organism
│   └── Organism
│       ├── id = taxon:7227
│       ├── name = Drosophila melanogaster
│       └── ncbi_taxonomy_id = taxon:7227
├── reference_sequence = NM_001300612.1
└── annotations
    ├── 0
    │   └── Annotation
    │       ├── id = annotation0
    │       ├── start_position = 14
    │       ├── end_position = 361
    │       ├── note = Fructose-bisphosphate aldolase class-I; pfam00274
    │       ├── name = Glycolytic
    │       └── db_xref = CDD:425574
    ├── 1
    │   └── Annotation


In [4]:
rec = pblast(seq.amino_acid_sequence, n_hits=100)

### Building the SQL database

Next, we are going to use the ```build_sql_database``` function to set up an SQLite database file. This one will be used later on to populate data from our model. The database will construct a tabe for each object/attribute and thus facilitate an easy transfer from an application to the database.

### Creating a dataset

In order to demonstrate how to populate the database using our model, we are going to construct a small dataset using the data model we just loaded.

### Populating the database

Finally, we are going to add the data to our previously created database by using the datasets ```to_sql``` method where we also submit the location of our database file. This will create an Object Relation Model (ORM) that represents the Database structure and map the values present in our dataset to the corresponding tables.

In [11]:
from sdRDM import DataModel
from sdRDM_db import DBConnector, create_tables

In [12]:
# Establish a connection to the database
db = DBConnector(
    username="root", password="root", host="localhost", db_name="ExampleDB"
)

In [14]:
db

DBConnector(host='localhost', username='root', password='root', db_name='ExampleDB', port=5432, dbtype=<SupportedBackends.POSTGRES: 'postgres'>, connection=<ibis.backends.postgres.Backend object at 0x1295b0910>)

In [18]:
# Check if there are no tables in the database
db

DBConnector(host='localhost', username='root', password='root', db_name='ExampleDB', port=5432, dbtype=<SupportedBackends.POSTGRES: 'postgres'>, connection=<ibis.backends.postgres.Backend object at 0x1295b0910>)